## Задание №11

Ваш знакомый Вася решил создать стартап и привлечь инвестиции. Для этого он хочет разместить информацию о своем стартапе на Producthunt — платформе, где создатели размещают информацию о своих стартапах, а пользователи за эти стартапы голосуют. Вася хочет, чтобы как можно больше пользователей проголосовали за его стартап. Поэтому он хочет понять, какие факторы влияют на то, сколько голосов получит та или иная идея на Producthunt. Для этого Вася попросил вас создать модель машинного обучения, которая по информации на странице стартапа предсказывала бы, сколько голосов этот стартап получит.

**Формат ввода**

Данные доступны по ссылке [https://disk.yandex.ru/d/RKfUnpuddXZ0vg](https://disk.yandex.ru/d/RKfUnpuddXZ0vg). Вам дана обучающая выборка `producthunt_train.csv` с информацией о стартапах. Целевая переменная `votesCount` содержит количество голосов, которое этот стартап получил.

**Формат вывода**

Вам необходимо для всех строк из тестовой выборки `producthunt_test.csv` предсказать количество голосов, которое получит соответствующий стартап. Файл `sample_submission.csv` содержит требуемый формат сдачи.

**Примечания**

Оценка решения будет производиться по метрике $R^2$ (коэффициент детерминации). Итоговый балл будет ставиться по формуле $\min(10, \max(0, score \cdot 20))$, где $score$ — значение метрики.

In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [23]:
cols_to_encode = ['slug', 'name', 'tagline', 'topics', 'dateAdded', 'timeAdded']
def encode_data(train_data: pd.DataFrame, test_data: pd.DataFrame, cols_to_encode: list):
    le = LabelEncoder()
    
    for col in cols_to_encode:
        train_data[col] = le.fit_transform(train_data[col])
        test_data[col] = le.fit_transform(test_data[col])
        
    return train_data, test_data

def split_data(train_data: pd.DataFrame, target_column: str):
    X_train = train_data.drop(columns=[target_column])
    Y_train = train_data[target_column]
    return X_train, Y_train

train_data = pd.read_csv('producthunt_train.csv')
test_data = pd.read_csv('producthunt_test.csv')

train_data, test_data = encode_data(train_data, test_data, cols_to_encode)

X_train, Y_train = split_data(train_data, 'votesCount')

### LinearRegression (R^2=0.283)

In [25]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, Y_train)

from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, Y_train, cv=5, scoring='r2')
average_score = scores.mean()

print(f"Средний R^2-коэффициент на кросс-валидации: {average_score}")

Средний R^2-коэффициент на кросс-валидации: 0.2831062591230047


### XGBRegressor (R^2=0.465)

In [26]:
# Бустинг
import xgboost as xgb

model = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators = 350)
model.fit(X_train, Y_train)

scores = cross_val_score(model, X_train, Y_train, cv=5, scoring='r2')
average_score = scores.mean()

print(f"Средний R^2-коэффициент на кросс-валидации: {average_score}")

Средний R^2-коэффициент на кросс-валидации: 0.4657581679510642


### XGBRFRegressor (R^2=0.483)

In [27]:
from sklearn.model_selection import GridSearchCV

model = xgb.XGBRFRegressor(objective='reg:squarederror', learning_rate=1)
cvParams = {
    'n_estimators': np.linspace(10, 500, 100, dtype=int),
}
cv = GridSearchCV(model, cvParams, n_jobs=16, cv=5)
cv.fit(X_train, Y_train)
model = cv.best_estimator_

scores = cross_val_score(model, X_train, Y_train, cv=5, scoring='r2')
average_score = scores.mean()
print(f"Средний R^2-коэффициент на кросс-валидации: {average_score}")

Средний R^2-коэффициент на кросс-валидации: 0.48300497199613074


### LGBMRegressor (R^2=0.514)

In [28]:
import lightgbm as lgb

lgb_model = lgb.LGBMRegressor(n_estimators=100)

lgb_model.fit(X_train, Y_train)

lgb_scores = cross_val_score(lgb_model, X_train, Y_train, cv=5, scoring='r2')
lgb_average_score = lgb_scores.mean()

print(f"Средний R^2-коэффициент на кросс-валидации для LightGBM: {lgb_average_score}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001247 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 56702, number of used features: 8
[LightGBM] [Info] Start training from score 287.430902
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000827 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 45361, number of used features: 8
[LightGBM] [Info] Start training from score 287.085823
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000902 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 45361, number of used features: 8
[LightGBM] [Info] Start tr

Для получения максимального балла необходимо набрать **R^2 > 0.5**\
Лучше всего показала себя модель **LightGBM** (**R^2=0.514**). Дальнейшее улучшение модели или обработка данных не требуется.

In [30]:
# Результаты LightGBM
votes_pred = lgb_model.predict(test_data)

submission = pd.DataFrame({
    "id": test_data["id"], 
    "votesCount": votes_pred
})
submission.to_csv('submission.csv', index=False)